In [178]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)


from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [187]:
# mesurePlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/article/9e/validation/pdd9e_mesure.csv"
# gatePlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/article/9e/validation/pdd9e.csv"

plan="in"

mesurePlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/article/9e/validation/9eldp_mesure_"+plan+".csv"
gatePlot="/media/sf_CRCT/JupyterNotebook/data/gammaIndex/article/9e/validation/prof9e_"+plan+".csv"

mesurePlotData=pd.read_csv(mesurePlot, sep=',')
gatePlotData=pd.read_csv(gatePlot, sep=',')

#print(mesurePlotData)

In [195]:
LinearInterpolation=True
#Point every (in mm)
#NewStep=0.1

Threshold=True
#Of Dmax
ThresholdValue=0.1


#Gamma Index criterion 
DTA=1 #mm
deltaDmax=0.02 #%


name='name'



In [196]:

xGscale=0.5


#Create numpy ndarray from pandas dataFrames
refX=mesurePlotData['pas']
refY=mesurePlotData['value']
gateX=gatePlotData['pas']
gateY=gatePlotData['value']




# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
    gateY2=np.interp(gateX2, gateX, gateY)


else:
    refX2= refX
    refY2= refY
    gateX2= gateX
    gateY2= gateY

# Centering LDP
gateX2=gateX2-(gateX.max()/2)
   
    
#normalisation au centre 
maskFilm=(refX2 > -2) & (refX2 < 2)
maskGate=(gateX2 > -2) & (gateX2 < 2)
refY2=refY2/refY2[maskFilm].mean()
gateY2=gateY2/gateY2[maskGate].mean()

#normalisation au max
# maxR=refY2.max()
# maxG=gateY2.max()
# refY2=refY2/maxR
# gateY2=gateY2/maxG

#res pixel
gateX2=gateX2*2.0

#SCALING
gateX2=gateX2+xGscale


#mask for remove useless tails

mask=(gateX2>-80) & (gateX2 < 80)
gateX2=gateX2[mask]
gateY2=gateY2[mask]


#create numpy ndarray for gamma comprisonj
deltaD=np.zeros(len(refY2))
gammaIndex=np.zeros(len(gateX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(gateX2)):
    for j in range(len(refX2)):
        dr[j]=(gateX2[i]-refX2[j])
        deltaD[j]=gateY2[i]-refY2[j]
        gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
    gammaIndex[i]=gammaIndexTEMP.min()

#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex)):

        if gateY2[i]>0.10:
            a=a+1
            if gammaIndex[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate=(nok/a)*100
else:
    nok=(gammaIndex < 1).sum()
    passRate=nok*100.0/len(gammaIndex)

#print results
percentDmax=deltaDmax*100
print('RESULTAT GATE : ')
print('*   Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm : '  +str(passRate) +' %')


#compute diff % 
#diffGate=(refY2-gateY2)*100


# Plot profil or PDD with plotly




nombre de points testés : 1173
nombre de points passant le test (<1) : 1076.0
RESULTAT GATE : 
*   Gamma Index Pass Rate 2.0% - 1mm : 91.7306052856 %


In [199]:
gate = Scatter(x = gateX2,
               y = gateY2,
               name ='gate',
               mode = 'lines')
mesure = Scatter(x = refX2,
                 y = refY2,
                 name ='mesure',
                 mode = 'lines')

gamma = Scatter(x = gateX2,
                 y = gammaIndex,
                 name ='gamma',
                 mode = 'line')

fig = dict(data=[mesure,gate,gamma])
iplot(fig, image='svg')


In [ ]:
#data=np.column_stack((gateX2, gateY2))
#np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/18e/validation/18epddmes1mm.csv', mes, delimiter=',')


In [169]:
mes=np.column_stack((refX2, refY2))
gate=np.column_stack((gateX2, gateY2))
gamma=np.column_stack((gateX2, gammaIndex))

# SAVE LDP 
# np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/18e/validation/prof18e_mesure_'+plan+'.csv', mes, delimiter=',')
# np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/18e/validation/prof18e_gate_'+plan+'.csv', gate, delimiter=',')
# np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/18e/validation/prof18e_indexG_'+plan+'.csv', gamma, delimiter=',')

# SAVE PDD
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/9e/validation/pdd9e_mesure2.csv', mes, delimiter=',')
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/9e/validation/pdd9e_gate2.csv', gate, delimiter=',')
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/2/9e/validation/pdd9e_indexG2.csv', gamma, delimiter=',')

In [ ]:
# plot PDD 
mesurePlot1="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/6E/new/validation/profil6eMesure.csv"
gatePlot1="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/6E/new/validation/profil6eGateWGood.csv"

mesurePlot2="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/9E/new/validation/profil9eMesure.csv"
gatePlot2="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/9E/new/validation/profil9eGateWGood.csv"

mesurePlotData1=pd.read_csv(mesurePlot1, sep=';')
gatePlotData1=pd.read_csv(gatePlot1, sep=',')
mesurePlotData2=pd.read_csv(mesurePlot2, sep=';')
gatePlotData2=pd.read_csv(gatePlot2, sep=',')

refX6=mesurePlotData1['pas']
refY6=mesurePlotData1['value']
refX9=mesurePlotData2['pas']
refY9=mesurePlotData2['value']
gateX6=gatePlotData1['pas']
gateY6=gatePlotData1['value']
gateX9=gatePlotData2['pas']
gateY9=gatePlotData2['value']


maxR6=refY6.max()
maxG6=gateY6.max()
refY6=refY6/maxR6
gateY6=gateY6/maxG6

maxR9=refY9.max()
maxG9=gateY9.max()
refY9=refY9/maxR9
gateY9=gateY9/maxG9





refX26=np.linspace(refX6.min(), refX6.max(), 2000)
refY26=np.interp(refX26, refX6, refY6)
refX29=np.linspace(refX9.min(), refX9.max(), 2000)
refY29=np.interp(refX29, refX9, refY9)

gateX2_6=np.linspace(gateX6.min(), gateX6.max(), 50)
gateY2_6=np.interp(gateX2_6, gateX6, gateY6)

gateX2_9=np.linspace(gateX9.min(), gateX9.max(), 50)
gateY2_9=np.interp(gateX2_9, gateX9, gateY9)

gateY2_9=gateY2_9*1.2
refY29=refY29*1.2

trace1 = go.Scatter(
    x = refX26,
    y = refY26,
    mode = 'line', 
    name = '6 MeV - measure', 
    line = dict(
        color = ('rgba(0, 0, 0, 100)'), 
        width = 1.5,
        dash = 'dot'
    )
)


trace2 = go.Scatter(
    x = refX29,
    y = refY29,
    mode = 'line', 
    name = '9 MeV - measure', 
    line = dict(
        color = ('rgba(0, 0, 0, 100)'), 
        width = 1.5
    )
)

trace3 = go.Scatter(
    x = gateX2_6,
    y = gateY2_6,
    mode = 'line+markers', 
    name = '6 MeV - GATE', 
    line = dict(
        color = ('rgba(0, 92, 245, 100)'),
        dash='dot', 
        width=1.5
    ), 
    marker = dict (
        size = 4, 
         color= ('rgba(255, 255, 255, 100)'),
    line= dict(
        color = ('rgba(0, 92, 245, 100)'),
        width = 1     
              ) 

                  )
)

trace4 = go.Scatter(
    x = gateX2_9,
    y = gateY2_9,
    mode = 'line+markers', 
    name = '9 MeV - GATE', 
    line = dict(
        color = ('rgba(0, 92, 245, 100)'),
        dash='dot', 
        width=1.5
    ), 
    marker = dict (
        size = 4, 
         color= ('rgba(228, 26, 26, 100)'),
                  )
)


#    #Plot gamma Index with plotly
# trace4 = go.Scatter(
#     x = gateX2,
#     y = gammaIndex,
#     mode = 'lines', 
#     name = 'Gamma Index GATE ', 
#     line = dict(
#         color = ('rgba(44, 160, 44, 100)'), 
#         dash='dot'
#     )
# )



layout1= go.Layout(
    #title='',
    #titlefont=dict(
    #size=24),
    xaxis=dict(
        range=[-100, 100],
        title='Off axis distance (mm)',
        titlefont=dict(
            size=20
        )
),

    yaxis=dict(
        title='Relative Dose',
        titlefont=dict(
            size=20)
    )
)


layout2= go.Layout(
    title='Gamma Index - 6 MeV',
    titlefont=dict(
    size=24),
    annotations=[dict(
        x=35,
        y=2,
        xref='gateX2',
        yref='gammaIndex',
        showarrow=False, 
        text='GIPR 2%-2mm: ' + '%',
        font=dict(
        size=18)
     )],
    xaxis=dict(
        title='Depth (mm)',
        range=[0,70],
        titlefont=dict(
                size=20)
    ),
    yaxis=dict(
        title='Gamma index value',
        range=[0, 3],
        titlefont=dict(
                size=20)
    ),

)


layout3= go.Layout(
    xaxis=dict(
        title='Off distance axis (mm)',
        range=[0,70],
        titlefont=dict(
            size=20)
),
    yaxis=dict(
        title='Difference %',
        #range=[-30, 30],
        titlefont=dict(
                size=20)
)
)


# trace6 = go.Scatter(
#     x = gateX2,
#     y = diffGate,
#     mode = 'lines', 
#     name = 'Difference % GATE ', 
#     line = dict(
#         color = ('rgba(44, 160, 44, 100)'), 
#         dash='dash'
#     )
# )



data1 = [trace1, trace2, trace3, trace4]
fig1= go.Figure(data=data1, layout=layout1)

py.iplot(fig1, filename= 'pddValidation69', image='svg') 



